In [14]:
import os
import json
import boto3
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Fetch inference profile ARN from environment variables
bedrock_inference_profile_arn = os.getenv("BEDROCK_INFERENCE_PROFILE_ARN")

# Initialize the Bedrock Runtime client
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_REGION")
)

def invoke_bedrock_model(profile_arn, payload):
    """
    Invoke a Bedrock model using the inference profile ARN.

    Args:
        profile_arn (str): ARN of the Bedrock inference profile.
        payload (dict): Input payload for the model.

    Returns:
        dict: Response from the Bedrock model.
    """
    try:
        logger.info(f"Invoking Bedrock model")
        
        response = bedrock_runtime.invoke_model(
            modelId=profile_arn,
            body=json.dumps(payload),
            contentType="application/json",
            accept="application/json"
        )
        
        # Parse and return the response
        response_body = json.loads(response["body"].read())
        return response_body

    except Exception as e:
        logger.error(f"Error invoking model with ARN {profile_arn}: {e}")
        return {"error": str(e)}

# Payload for the smart home agent's winter preparation task
smart_home_payload = {
    "prompt": """Prepare the house for winter by optimizing thermostat, plumbing, lighting, garage, and garden. 
Ensure pipes won't freeze and all systems are secure.""",
    "temperature": 0.7,
    "top_p": 0.9
}

# Call the Bedrock model
try:
    smart_home_response = invoke_bedrock_model(bedrock_inference_profile_arn, smart_home_payload)
    
    # Print the raw response
    print("Smart Home Agent Response:")
    print(json.dumps(smart_home_response, indent=2))

    # Extract and print generated instructions
    generated_text = smart_home_response.get("generation", "No generation found")
    print("\nGenerated Instructions:\n", generated_text)

except Exception as e:
    print(f"Error: {e}")

INFO:__main__:Invoking Bedrock model with ARN: arn:aws:bedrock:us-east-2:221777993733:inference-profile/us.meta.llama3-2-3b-instruct-v1:0


Smart Home Agent Response:
{
  "generation": " \nCheck the roof and gutters for damage and clean them out to prevent water from accumulating and causing damage. \nCheck all doors and windows for proper sealing to prevent air leaks and heat loss. \nUse the correct type of window treatments to keep warm air in and cold air out. \nPrevent moisture buildup in the walls and ceilings by checking for any signs of water damage or leaks. \nKeep the garage door closed during the winter to prevent cold air from entering the house. \nUse a programmable thermostat to save energy and stay warm. \nUse zone heating to warm only the rooms you use most, and turn off the heat in unused rooms. \nSeal any gaps or cracks in the walls and windows to prevent cold air from entering. \nUse thermal curtains or drapes to keep warm air in and cold air out. \nKeep the chimney clean and clear to ensure proper draft. \nCheck the smoke and carbon monoxide detectors to ensure they are working properly. \nCheck the fire